In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm
tqdm.pandas()
import re

In [6]:
path = '/home/sergey/Python_projects/RU_NER/Data/CommonVoice/Converted_Common_Voice_ru.csv'

In [7]:
df = pd.read_csv(path, index_col=0, low_memory=False)

In [8]:
df

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,18849003,Владимир вытащил пробку.,владимир вытащил пробку,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
1,18849004,Слово имеет уважаемый представитель Республики...,слово имеет уважаемый представитель в республи...,"['в', 'корее', 'корея', 'республике', 'республ...",validated,2,0,twenties,male,NaN,NaN,ru,NaN
2,18849005,Совет Безопасности приступает к рассмотрению п...,совет безопасности приступает к рассмотрению п...,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
3,18849006,Не лезь!,не лезь,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
4,18849007,И даже смеяться перестали.,даже смеяться перестали,['и'],validated,2,0,twenties,male,NaN,NaN,ru,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148637,37271921,Слово предоставляется Постоянному наблюдателю ...,слово предоставляется постоянного наблюдателя ...,"['наблюдателю', 'наблюдателя', 'постоянного', ...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN
148638,37271997,"С этим пледом я люблю посиживать на балконе, а...",с этим пледом я люблю посиживать на балконе а ...,['—'],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN
148639,37277285,О чем ты?,о чем ты,[],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN
148640,37285119,"Почвы по-прежнему загрязнены, но проекты лесоп...",почва по-прежнему загрязнена но проекта лесопо...,"['загрязнена', 'загрязнены', 'лесопользование'...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN


In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [73]:
test_sentence = df['sentence'][1]

In [74]:
ner_results = nlp(test_sentence)
print(ner_results)

[{'entity_group': 'LOC', 'score': 0.99750185, 'word': 'Республики Корея', 'start': 36, 'end': 52}]


In [79]:
df['NER'] = df['sentence'].progress_apply(lambda x: nlp(x))

100%|█████████████████████████████████| 148094/148094 [1:54:16<00:00, 21.60it/s]


In [ ]:
df.to_csv('/home/sergey/Python_projects/RU_NER/Data/CommonVoice/NER_Common_Voice_ru.csv')

##### part2:

In [1369]:
df = pd.read_csv('/home/sergey/Python_projects/RU_NER/Data/NER_Common_Voice_ru.csv', index_col=0, low_memory=False)

In [1371]:
markup = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [1372]:
df['NER'] = df['NER'].apply(literal_eval)

In [1373]:
score_treshold = 0.65
misc_treshold = 0.9

In [1374]:
# Concatenate strange tokenization (В + ##енесуэла = Венесуэла), and change the NER results accordingly,
# changing start/end according to new tokenization, as well as using mean score for combined entity
def ner_concat(sentence, ner):
    del_index = []
    merged_ner = []
    
    for index in range(len(ner)):
        if '#' not in ner[index]['word']:
            merged_ner.append(ner[index])
        else:
            ner[index]['word'] = ner[index]['word'].strip('#')
            if merged_ner and ner[index]['start'] == merged_ner[-1]['end']:
                merged_ner[-1]['word'] += ner[index]['word']
                merged_ner[-1]['score'] = np.mean([merged_ner[-1]['score'], ner[index]['score']])
                merged_ner[-1]['end'] = ner[index]['end']
            else:
                merged_ner.append(ner[index])
                
    tokens = re.sub(r'[^\w\s]', '', sentence).split()
    
    for item in merged_ner:
        remove_flag = False
        for word in re.sub(r'[^\w\s]', '', item['word']).split():
            if word not in tokens:
                remove_flag = True
        if remove_flag:
            merged_ner.remove(item)

    return merged_ner

In [1380]:
df['NER'] = df.apply(lambda row: ner_concat(row['sentence'], row['NER']), axis=1)

In [1368]:
df

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment,NER
0,18849003,Владимир вытащил пробку.,владимир вытащил пробку,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'PER', 'score': 0.9800292, '..."
1,18849004,Слово имеет уважаемый представитель Республики...,слово имеет уважаемый представитель в республи...,"['в', 'корее', 'корея', 'республике', 'республ...",validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'LOC', 'score': 0.99750185, ..."
2,18849005,Совет Безопасности приступает к рассмотрению п...,совет безопасности приступает к рассмотрению п...,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'ORG', 'score': 0.9949068, '..."
3,18849006,Не лезь!,не лезь,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.65403444,..."
4,18849007,И даже смеяться перестали.,даже смеяться перестали,['и'],validated,2,0,twenties,male,NaN,NaN,ru,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148637,37271921,Слово предоставляется Постоянному наблюдателю ...,слово предоставляется постоянного наблюдателя ...,"['наблюдателю', 'наблюдателя', 'постоянного', ...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.32757148,..."
148638,37271997,"С этим пледом я люблю посиживать на балконе, а...",с этим пледом я люблю посиживать на балконе а ...,['—'],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[]
148639,37277285,О чем ты?,о чем ты,[],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.84900314,..."
148640,37285119,"Почвы по-прежнему загрязнены, но проекты лесоп...",почва по-прежнему загрязнена но проекта лесопо...,"['загрязнена', 'загрязнены', 'лесопользование'...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[]


In [1376]:
# Remove bizzare results, which mostly have low score (threshold)
# using higher threshold for defined groups (LOC/PERS/ORG)
def ner_cleanup(ner):
    del_index = []
    for index in range(len(ner)):
        if re.sub(r'[^\w\s]', '', ner[index]['word']) == '' : del_index.append(index)
        elif ner[index]['score'] < score_treshold : del_index.append(index)
        elif (ner[index]['entity_group'] == 'MISC' and ner[index]['score'] < misc_treshold):
            del_index.append(index)
    for ele in sorted(del_index, reverse=True):
        del ner[ele]
        
    return ner

In [1377]:
df['NER'] = df['NER'].apply(lambda x: ner_cleanup(x))

In [1378]:
# Get a markup similar to other datasets, produces a list according to 'markup' for each sentence

def convert_markup(sentence, ner):
    tokens = re.sub(r'[^\w\s]', '', sentence).split()
    markup_list = [0] * len(tokens)
    
    for dic in ner:
        entity_start = dic['start']
        entity_end = dic['end']
        entity_type = dic['entity_group']
        entity_tokens = re.sub(r'[^\w\s]', '', sentence[entity_start:entity_end]).split()
        entity_length = len(entity_tokens)
        if entity_length == 1:
            markup_index = re.sub(r'[^\w\s]', '', sentence).split().index(entity_tokens[0])
            markup_list[markup_index] = markup['B-' + entity_type]
            
        else:
            markup_index = [re.sub(r'[^\w\s]', '', sentence).split().index(token) for token in entity_tokens]
            markup_list[markup_index[0]] = markup['B-' + entity_type]
            markup_list[markup_index[1]:markup_index[0]+entity_length] = [markup['I-' + entity_type] for _ in range(entity_length-1)]
            
    return markup_list

In [1381]:
df['NER_markup'] = df.apply(lambda row: convert_markup(row['sentence'], row['NER']), axis=1)

In [1383]:
df

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment,NER,NER_markup
0,18849003,Владимир вытащил пробку.,владимир вытащил пробку,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'PER', 'score': 0.9800292, '...","[1, 0, 0]"
1,18849004,Слово имеет уважаемый представитель Республики...,слово имеет уважаемый представитель в республи...,"['в', 'корее', 'корея', 'республике', 'республ...",validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'LOC', 'score': 0.99750185, ...","[0, 0, 0, 0, 5, 6]"
2,18849005,Совет Безопасности приступает к рассмотрению п...,совет безопасности приступает к рассмотрению п...,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'ORG', 'score': 0.9949068, '...","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
3,18849006,Не лезь!,не лезь,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN,[],"[0, 0]"
4,18849007,И даже смеяться перестали.,даже смеяться перестали,['и'],validated,2,0,twenties,male,NaN,NaN,ru,NaN,[],"[0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148637,37271921,Слово предоставляется Постоянному наблюдателю ...,слово предоставляется постоянного наблюдателя ...,"['наблюдателю', 'наблюдателя', 'постоянного', ...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[],"[0, 0, 0, 0, 0, 0, 0]"
148638,37271997,"С этим пледом я люблю посиживать на балконе, а...",с этим пледом я люблю посиживать на балконе а ...,['—'],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
148639,37277285,О чем ты?,о чем ты,[],validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[],"[0, 0, 0]"
148640,37285119,"Почвы по-прежнему загрязнены, но проекты лесоп...",почва по-прежнему загрязнена но проекта лесопо...,"['загрязнена', 'загрязнены', 'лесопользование'...",validated,3,0,thirties,male,"Акцент центрального черноземья,Обычный",NaN,ru,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [1385]:
df = df.drop_duplicates(subset='sentence', ignore_index=True)

In [1388]:
df.to_csv('Edited_NER_anotation_Common_Voice.csv')